<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/20_pct_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

12. 企業財務データの多変量解析：主成分分析

　本例では、東証上場企業の2021年決算データを用い、主成分分析による分析事例を示します。具体的には、上場企業の6つの財務指標データを集約して、その集約された主成分得点が多い企業がどの企業なのかを検証する手順を示します。

In [ ]:
#[1]ライブラリ読み込み
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.decomposition import PCA

:

財務データのcsvファイルを読み込みます。

In [ ]:
#[2]企業財務データ(CSVファイル)の読み込み
from google.colab import files
uploaded = files.upload()

読み込んだCSVファイルを今度はpandasデータフレームの形式に転換します。

In [ ]:
#[3]データフレームの作成
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['jpn_firm_data2021.csv']))
df.head()
df2=df.drop('id',axis=1)
df2

各財務指標の分布が多様であるため、標準偏差を共通化するデータの標準化を実施します。

In [ ]:
#[4]標準化
df2= df2.iloc[:, :].apply(lambda x: (x-x.mean())/x.std(), axis=0)
df2.head()
df2.describe()

主成分分析を実施します。

In [ ]:
#[5]主成分分析の実行
from numpy.core.fromnumeric import searchsorted
pca = PCA()
pca.fit(df2)
score= pca.transform(df2)
score=pd.DataFrame(score,columns=["PC{}".format(x + 1) for x in range(len(df2.columns))])
score.insert(0, 'id', df['id'])
score

基本的には以降の演算もpythonで進めた方がよいですが、Excelに保存する場合には下記のコードを実行します。

In [ ]:
#[6]標本企業別主成分得点の書き出し
from google.colab import files
score.to_csv('score.csv')
files.download('score.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

寄与率、累積寄与率を計算して、各主成分が、全財務データをどの程度まで説明できているのかを定量的に確認します。

In [ ]:
#[7]寄与率の算出: numpyの場合
pca.explained_variance_ratio_

In [ ]:
#[8]寄与率の算出: pandasの場合
pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(len(df2.columns))])

In [ ]:
#[9]累積寄与率の算出: numpyの場合
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
#[10]累積寄与率の算出: pandasの場合
import matplotlib.ticker as ticker
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.plot([0] + list( np.cumsum(pca.explained_variance_ratio_)), "-o")
plt.xlabel("Number of principal components")
plt.ylabel("Cumulative contribution rate")
plt.grid()
plt.show()

最後に因子負荷量を算出して、それぞれの「主成分」が①収益性、②安全性、③成長性、④その他、のどれを意味しているのかを確認します。

In [ ]:
#[11]]因子負荷量の算出： numpyの場合
pca.components_

In [ ]:
#[12]]因子負荷量の算出： pandasの場合
pd.DataFrame(pca.components_, columns=df.columns[1:], index=["PC{}".format(x + 1) for x in range(len(df2.columns))])